In [25]:
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
import numpy as np
from functions import *

In [28]:
class logi:
    def __init__(self,x):
        self.x=np.linspace(0,1,x)
    
    def logistic(self,x,r):
        return r*x*(1-x)
        
    def __call__(self,r):
        plt.close()
        y=self.logistic(self.x,r)
        plt.plot(self.x,y)
        plt.plot(np.array([0,1]),np.array([0,1]))
        plt.show()

In [32]:
x=30
f=logi(x)
r=widget.FloatLogSlider(value=0,min=-1,max=1)
interact(f.__call__,r=r)

interactive(children=(FloatLogSlider(value=0.1, description='r', max=1.0, min=-1.0), Output()), _dom_classes=(…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [26]:
plt.close()